In [21]:
import sys
import os
from scipy.sparse import load_npz
sys.path.append('./codes/forgraph/')
from config import args
from sklearn.metrics import roc_auc_score
from models import GCN
from metrics import *
import numpy as np
from Explainer import Explainer
import torch_geometric
from scipy.sparse import coo_matrix,csr_matrix
import pickle as pkl
import csv
import pandas as pd
import bokeh.io
import torch
import bokeh.plotting
bokeh.io.output_notebook()

FileNotFoundError: Could not find module 'C:\Users\whitn\anaconda3\Lib\site-packages\torch_sparse\_convert_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.

In [18]:
def acc(adj,insert):
    mask = explainer.masked_adj.numpy()
    adj = coo_matrix(adj)
    for r,c in list(zip(adj.row,adj.col)):
        if r>=insert and r<insert+skip and c>=insert and c<insert+skip:
            reals.append(1)
        else:
            reals.append(0)
        preds.append(mask[r][c])
        
def explain_graph(gid):
    fea,emb,adj,label,graphid = features[gid], embs[gid], adjs[gid], labels[gid], gid
    explainer((fea,emb,adj,1.0,label))
    insert = 20
    acc(adj,insert)
    
def test():
    global preds
    global reals
    preds = []
    reals = []
    for gid in allnodes:
        explain_graph(gid)
    auc = roc_auc_score(reals,preds)
    return auc

def train(epochs):
    t0 = args.coff_t0
    t1 = args.coff_te

    for epoch in range(epochs): 
        loss = 0
        tmp = float(t0 * np.power(t1 / t0, epoch /epochs))
        train_instances = [ins for ins in range(len(model.values()))]
        np.random.shuffle(train_instances)
        for gid in train_instances:
            with tf.GradientTape() as tape:
                pred = explainer((features[gid],embs[gid],adjs[gid],tmp, labels[gid]),training=True)
                loss += explainer.loss(pred, labels[gid])
        train_variables = [para for para in explainer.trainable_variables]
        grads = tape.gradient(loss, train_variables)
        optimizer.apply_gradients(zip(grads, train_variables))

In [19]:
file = 'actual data/src/data/E116/saved_runs/model_2020-12-17-at-19-24-36.pt'
#file = 'actual data/src/data/E116/df_genes.pkl'
#model = torch()
#model.load_state_dict(torch.load(file))
model = torch.load(file, map_location=torch.device('cpu'))

In [20]:
cell_line = 'E116'
base_path = 'actual data/src/data'
hic_sparse_mat_file = os.path.join(base_path, cell_line, 'hic_sparse.npz')
np_nodes_lab_genes_file = os.path.join(base_path, cell_line, 'np_nodes_lab_genes.npy')
np_hmods_norm_all_file = os.path.join(base_path, cell_line, 'np_hmods_norm.npy') 
df_genes_file = os.path.join(base_path, cell_line, 'df_genes.pkl')
df_genes = pd.read_pickle(df_genes_file)

###Define model inputs
num_feat = 5
num_classes = 2

mat = load_npz(hic_sparse_mat_file)
allNodes_hms = np.load(np_hmods_norm_all_file)
hms = allNodes_hms[:, 1:] #only includes features, not node ids
allNodes = allNodes_hms[:, 0].astype(int)
geneNodes_labs = np.load(np_nodes_lab_genes_file)
geneNodes = geneNodes_labs[:, -2].astype(int)
geneLabs = geneNodes_labs[:, -1].astype(int)

allLabs = 2*np.ones(np.shape(allNodes))
allLabs[geneNodes] = geneLabs

targetNode_mask = torch.tensor(geneNodes).long()
X = torch.tensor(hms).float().reshape(-1, 5)
Y = torch.tensor(allLabs).long()

extract = torch_geometric.utils.from_scipy_sparse_matrix(mat)
data = torch_geometric.data.Data(edge_index = extract[0], edge_attr = extract[1], x = X, y = Y)
G = data
num_feat = 5

NameError: name 'torch_geometric' is not defined

In [ ]:
features = hms
adjs = allNodes
labels = allLabs

In [ ]:
model = GCN(input_dim=features.shape[-1], output_dim=labels.shape[1])
model.load_weights(args.save_path+args.dataset)


embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)

acc = accuracy(outputs, labels)
print('model acc',acc.numpy())

embs = model.getNodeEmb((tf.convert_to_tensor(features,dtype=tf.float32),\
                               tf.convert_to_tensor(adjs,dtype=tf.float32)), training=False)
outputs = model.call((tf.convert_to_tensor(features,dtype=tf.float32),\
                            tf.convert_to_tensor(adjs,dtype=tf.float32)),training=False)
pred_label = tf.argmax(outputs, 1)

In [ ]:
if args.setting==1:
    allnodes = [i for i in range(0,100)]
elif args.setting==2:
    allnodes = [i for i in range(0,100)]
    allnodes.extend([i for i in range(500,600)])
elif args.setting==3:
    allnodes=[i for i in range(1000)]
explainer = Explainer(model=model,nodesize=len(model.values()))

In [ ]:
top_k = [i for i in range(0, 26)]
all_elr = [i for i in range(0, 26)]
coff_t0 = [i for i in range(1, 26)]
coff_size = [i for i in range(0, 26)]
coff_ent = [i for i in range(0, 26)]
skp = [i for i in range(0, 26)]
epochs = [i for i in range(250, 650, 50)]

In [ ]:
epochs

In [4]:
rows = []

In [5]:
for k in top_k:
    topk = k
    for elr in all_elr:
        args.elr = elr
        optimizer = tf.keras.optimizers.Adam(learning_rate=args.elr)
        for c in coff_t0:
            args.coff_t0 = c
            for s in coff_size:
                args.coff_size = s
                for en in coff_ent:
                    args.coff_ent = en
                    for sk in skp:
                        skip = sk
                        for ep in epochs:
                            train(ep)
                            auc = test()
                            row = [k, elr, c, en, sk, ep, auc]
                            rows.append(row)

NameError: name 'top_k' is not defined

In [6]:
col_names = ['K', 'elr', 'coff_t0', 'coff_size', 'coff_ent', 'Skip', 'AUC']
filename = 'results.csv'
with open('results.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(col_names) 
    write.writerows(rows) 

In [7]:
df = pd.read_csv(file_name)
df.head()

NameError: name 'file_name' is not defined

In [8]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="K",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="elr",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="coff_t0",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="coff_size",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="coff_ent",
    y_axis_label="AUC",
)
bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="Skip",
    y_axis_label="AUC",
)
bokeh.io.show(p)